In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join("../cnwi"))
sys.path.insert(0, module_path)

# Toy RF Pipeline

In [23]:
import ee
import geemap

In [24]:
ee.Initialize()

In [35]:
import cfg
import inputs
import td
import rf
import landcovers as lc

## Generate Input Stack

In [26]:
viewport = ee.FeatureCollection("users/ryangilberthamilton/BC/williston/williston_sub_a_2019")
wil_dc_stack = inputs.Williston_Data_Cube_Stack(
    viewport=viewport
)
# wil_dc_stack

## Create Training Data and Generate Samples

In [27]:
table = ee.FeatureCollection("users/ryangilberthamilton/BC/williston/fpca/willistonA_no_floodplain")

training_data = td.TrainingData(
    collection = table
)
training_data.label = 'cDesc'

td.generate_samples(
    stack = wil_dc_stack,
    training_data = training_data
)

# training_data.samples.first()


TrainingData(collection=<ee.featurecollection.FeatureCollection object at 0x000001F831902080>, label='cDesc', value='value')

In [31]:
rf_config = rf.RandomForestCFG(
    stack = wil_dc_stack,
    training_data = training_data
)
rf_config.predictors

In [32]:
random_forest = rf.cnwi_random_forest(
    config = rf_config
)

In [33]:
random_forest

RFOutput(model=<ee.Classifier object at 0x000001F831A48D60>, classified=<ee.image.Image object at 0x000001F831A49FC0>, samples=TrainingData(collection=<ee.featurecollection.FeatureCollection object at 0x000001F831902080>, label='cDesc', value='value'))

In [34]:
random_forest.classified

In [38]:

labels = random_forest.samples.labels
lc_obj = lc.LandCovers(labels=labels)
lc_obj.colours

{'BOG': 'A52A2A',
 'FEN': 'FFF600',
 'MARSH': '8FBC8F',
 'SWAMP': '008000',
 'UPLAND': 'FF0001',
 'WATER': '0000FF'}

In [39]:
Map = geemap.Map()
vis = {'palette': lc_obj.palette, 'min': 1, 'max': 6}

Map.addLayer(random_forest.classified, vis, "Classification")

Map.centerObject(table, 12)
Map

Map(center=[54.63038654400906, -122.11326989291922], controls=(WidgetControl(options=['position', 'transparent…